In [15]:
import Random
rng = Random.MersenneTwister();

import ForwardDiff as fd
import ReverseDiff as rd

import LinearAlgebra as la

using BenchmarkTools: @btime

include("NeuralNetwork/layer.jl")
include("NeuralNetwork/neuralnetwork.jl")

gradient (generic function with 1 method)

In [2]:
positions = rand(rng, Float64, 5)

5-element Vector{Float64}:
 0.3497433632661675
 0.47180459762210436
 0.9439212529316561
 0.653047357468389
 0.8233468487153084

## Neural Network

In [14]:
@time nn = NeuralNetwork([Dense((5, 5), rng),
    Dense((3, 5), rng),
    Dense((1, 3), rng),
    Exp(),
    DeArray()], positions);

  0.000192 seconds (379 allocations: 24.438 KiB)


In [9]:
@time model(positions, nn)

  0.000021 seconds (8 allocations: 560 bytes)


190.24024647432344

In [10]:
@time model_flat(positions, nn, nn.params_flat)

  0.000043 seconds (36 allocations: 2.719 KiB)


190.24024647432344

In [11]:
@time kinetic(positions, nn)

  0.000071 seconds (95 allocations: 11.875 KiB)


-1300.2633090524791

In [12]:
@time gradient(positions, nn);

  0.000026 seconds (2 allocations: 1.016 KiB)


In [8]:
grad_result = zeros(length(positions), length(positions));
grad_config = rd.HessianConfig(positions)
grad_tape = rd.HessianTape(positions -> model(positions, nn), positions, grad_config)
grad_tape = rd.compile(grad_tape)

typename(ReverseDiff.CompiledTape)(#9)

In [10]:
@time rd.hessian!(grad_result, grad_tape, positions)

  0.000052 seconds


5×5 Matrix{Float64}:
 248.103  265.442  172.092  169.529  283.394
 265.442  283.993  184.119  181.377  303.199
 172.092  184.119  119.369  117.591  196.571
 169.529  181.377  117.591  115.839  193.643
 283.394  303.199  196.571  193.643  323.704